In [1]:
import csv
from collections import defaultdict
import math
import pprint as pp
import numpy as np
import os

In [13]:
def rfid_info(filename):
    rfids = defaultdict(list)
    with open(filename) as f:
        csv_reader = csv.reader(f, delimiter=',')
        line_count = 0

        rfid_start = 1 << 64
        rfid_end = 0
        for row in csv_reader:
            epc = row[0] # electronic product code
            peakRssiInDbm = float(row[1]) # Received Signal Strength Indicator
            phase_diff1 = float(row[2]) # Phase difference between [WHAT]
            rfDopplerFreq = row[3] # 
            channelMhz = float(row[4]) # Which channel it was on
            time_millis = float(row[5]) / 1000
            rfid_start = min(rfid_start, float(time_millis))
            rfid_end = max(rfid_end, float(time_millis))
            data = {'peak_rssi': peakRssiInDbm, 'phase_diff': phase_diff1, 'rfDopplerFreq': rfDopplerFreq, 'channelMhz': channelMhz, 'time_millis': time_millis}
            rfids[epc].append(data)
    return rfids

def select_useful_rfid(rfid_info):
    return rfid_info['E20038EBB5953DC9B7693E9C']

def rfid_avgs(rfids):
    avg_rfids_rev = {k: sum([x['peak_rssi'] for x in v]) / len(v) for k, v in rfids.items()}
    return avg_rfids_rev

def wavelength(freq_mhz):
    return 3 * (10 ** 2) / freq_mhz
        
def rfid_dists(measurements):
    epc_dists = {}
    dists = []
    dist = 0 # might need to change to initial phase
    for measurement in measurements:
        wave_len = wavelength(measurement['channelMhz'])
        dists.append(dist)
        dist += measurement['phase_diff'] % (2 * math.pi) * wave_len
        epc_dists[epc] = dists
    
    return epc_dists

def img_metadata(filename):
    timestamps = [float(time) for time in open(filename, 'r').readlines()]
    return timestamps

# Array is sorted
def idx_find(array, target, start=0):
    if start >= len(array):
        return -1
    for idx, val in enumerate(array):
        if idx >= start:
            if val >= target:
                return idx - 1
    return -1

def sanity_check(img_times, tag_times, img2tag):
    diffs = np.zeros((len(img_times), len(tag_times)))
    for img_idx, img_time in enumerate(img_times):
        for tag_idx, tag_time in enumerate(tag_times):
            diffs[img_idx, tag_idx] = abs(tag_time - img_time)
    
    wrong = False
    min_diffs = np.min(diffs, axis=1)
    for img_idx, img_time in enumerate(img_times):
        smallest_tag_idx = np.argmin(diffs[img_idx, :])
        smallest_tag = tag_times[smallest_tag_idx]
        if smallest_tag not in img2tag[img_time]:
            print('img_time:', img_time, 'is wrong.', 'Should be {:.3f}, but got {}'.format(smallest_tag, img2tag[img_time]))
            wrong = True
    
    if not wrong:
        print('All good!')

In [14]:
def bound_data(tag_times, img_times):
    if tag_times[0] < img_times[0]:
        tag_start = idx_find(tag_times, img_times[0])
        img_start = 0
    else:
        tag_start = 0
        img_start = idx_find(img_times, tag_times[0])
        
    if tag_times[-1] < img_times[-1]:
        tag_end = len(tag_times) - 1
        img_end = idx_find(img_times, tag_times[-1])
    else:
        tag_end = idx_find(tag_times, img_times[-1])
        img_end = len(img_times) - 1
    
    return tag_start, tag_end, img_start, img_end
    

def bucketize(bucket_size, tag_times, img_times):
    tag_start, tag_end, img_start, img_end = bound_data(tag_times, img_times)
    
    all_buckets_start = min(tag_times[tag_start], img_times[img_start]) - bucket_size / 2
    all_buckets_end = max(tag_times[tag_end], img_times[img_end]) + bucket_size / 2
    num_buckets = math.ceil((all_buckets_end - all_buckets_start) / bucket_size)
    bucket_start_times = [all_buckets_start + bucket_size * i for i in range(0, num_buckets)]
    buckets = [([], []) for i in range(len(bucket_start_times))] # list of tuples of lists
    
    tag_pointer = tag_start
    cur_tag_value = tag_times[tag_pointer]
    bucket_pointer = 0
    while tag_pointer < tag_end and bucket_pointer < len(buckets) - 1:
        if not bucket_start_times[bucket_pointer] <= cur_tag_value < bucket_start_times[bucket_pointer + 1]:
            bucket_pointer += 1
            
        buckets[bucket_pointer][0].append(tag_times[tag_pointer] * 1000)
        tag_pointer += 1
        cur_tag_value = tag_times[tag_pointer]

    img_pointer = img_start
    bucket_pointer = 0
    while img_pointer < img_end and bucket_pointer < len(buckets) - 1:
        if not bucket_start_times[bucket_pointer] <= img_times[img_pointer] < bucket_start_times[bucket_pointer + 1]:
            bucket_pointer += 1
            
        buckets[bucket_pointer][1].append(img_times[img_pointer])
        img_pointer += 1
    
    return buckets

def bucket_stats(buckets):
    bucket_sizes = defaultdict(int)
    
    for tag_bucket, img_bucket in buckets:
        bucket_sizes[f'tag_size_{len(tag_bucket)}'] += 1
        bucket_sizes[f'img_size_{len(img_bucket)}'] += 1
        bucket_sizes[f'all_size_{len(tag_bucket) + len(img_bucket)}'] += 1
    
    return bucket_sizes

In [4]:
experiment_name = 'stationary_center'
rfid_data = rfid_info(f'data/{experiment_name}/tag_data.csv')
img_data = img_metadata(f'data/{experiment_name}/{experiment_name}_meta.txt')
disc1_tag = select_useful_rfid(rfid_data)
disc_time_data = [tag_data['time_millis'] for tag_data in disc1_tag]

In [15]:
# We create buckets of the same size across ALL readings and images
# So care is needed to determine the bucket size. We want to have a bucket to have 

bucket_size = 0.04

for root, dirs, files in os.walk('data/'):
    for experiment_name in dirs:
        experiment_dir = os.path.join(root, experiment_name)
        try:
            rfid_data = rfid_info(os.path.join(experiment_dir, 'tag_data.csv'))
        except:
            print(f'{experiment_dir} does not have tag_data.csv')
            print()
            continue
        
        try:
            img_data = img_metadata(os.path.join(experiment_dir, f'{experiment_name}_meta.txt'))
        except:
            print(f'{experiment_dir} does not have _meta.txt')
            print()
            continue
            
        disc_tag = select_useful_rfid(rfid_data)
        disc_time_data = [tag_data['time_millis'] for tag_data in disc_tag]
        buckets = bucketize(bucket_size, disc_time_data, img_data)
        for tag_times, img_times in buckets:
            print(tag_times, img_times)
        all_stats = bucket_stats(buckets)

#         for stat_name, count in sorted(all_stats.items()):
#             if not stat_name.startswith('all'):
#                 print(f'{stat_name}: {count}')
#         print()

[1541464550747.0] []
[1541464550870.0] [1541464550.773688, 1541464550.78302, 1541464550.7901828, 1541464550.796187, 1541464550.8022988]
[1541464550872.0] [1541464550.843502]
[1541464550874.0] [1541464550.88505]
[1541464550994.0] [1541464550.940804]
[1541464550996.0] [1541464550.972042]
[1541464550998.0] [1541464551.0100522]
[1541464551073.0] [1541464551.038336]
[1541464551075.0] [1541464551.1089559]
[1541464551201.0] [1541464551.138463]
[1541464551205.0] [1541464551.170088]
[1541464551208.0] [1541464551.210453]
[1541464551248.0] [1541464551.268345]
[1541464551352.0] [1541464551.305446]
[1541464551355.0] [1541464551.333174]
[1541464551357.0] [1541464551.371197]
[1541464551448.0] [1541464551.4340749]
[1541464551449.0] [1541464551.4817338]
[1541464551450.0] [1541464551.503892]
[1541464551511.0] [1541464551.575236]
[1541464551616.0] [1541464551.612174]
[1541464551618.0] [1541464551.644119]
[1541464551619.0] [1541464551.677096]
[1541464551753.0] [1541464551.741982]
[1541464551757.0] [154146

[1541465542200.0] [1541465541.161185]
[1541465542289.0] [1541465541.2024362]
[1541465542363.0] [1541465541.23239]
[1541465542365.0] [1541465541.30246]
[1541465542447.0] [1541465541.3341389]
[1541465542448.0] [1541465541.363816]
[1541465542526.0] [1541465541.401752]
[1541465542616.0] [1541465541.430569]
[1541465542617.0] [1541465541.486933]
[1541465542618.0] [1541465541.530449]
[1541465542751.0] [1541465541.557389]
[1541465542752.0] [1541465541.5853639]
[1541465542752.0] [1541465541.653286]
[1541465542851.0] [1541465541.690261]
[1541465542852.0] [1541465541.720362]
[1541465543000.0] [1541465541.763438]
[1541465543602.0] [1541465541.789322]
[1541465543685.0] [1541465541.849977]
[1541465543686.0] [1541465541.894272]
[1541465543769.0] [1541465541.927855]
[1541465543771.0] [1541465541.954008]
[1541465543846.0] [1541465541.9871511]
[1541465543847.0] [1541465542.0568142]
[1541465543920.0] [1541465542.085946]
[1541465543921.0] [1541465542.1244328]
[1541465544037.0] [1541465542.155207]
[1541465

[1541801634341.0] [1541801628.7120771]
[1541801634466.0] [1541801628.7522979]
[1541801634467.0] [1541801628.810984]
[1541801634467.0] [1541801628.851219]
[1541801634533.0] [1541801628.877056]
[1541801634534.0] [1541801628.9176588]
[1541801634637.0] [1541801628.976379]
[1541801634638.0] [1541801629.01439]
[1541801634638.0] [1541801629.043479]
[1541801634640.0] [1541801629.082507]
[1541801634762.0] [1541801629.1421282]
[1541801634762.0] [1541801629.1814048]
[1541801634763.0] [1541801629.2107441]
[1541801634881.0] [1541801629.238667]
[1541801634882.0] [1541801629.308328]
[1541801634883.0] [1541801629.346091]
[1541801634948.0] [1541801629.376182]
[1541801634948.0] [1541801629.4016771]
[1541801635033.0] [1541801629.4694731]
[1541801635113.0] [1541801629.5084748]
[1541801635113.0] [1541801629.536736]
[1541801635114.0] [1541801629.604002]
[1541801635212.0] [1541801629.646539]
[1541801635212.0] [1541801629.673221]
[1541801635213.0] [1541801629.697685]
[1541801635213.0] [1541801629.768318]
[154

[1541801700391.0] [1541801680.687522]
[1541801700391.0] [1541801680.72493]
[1541801700393.0] [1541801680.7541492]
[1541801700482.0] [1541801680.825415]
[1541801700483.0] [1541801680.855222]
[1541801700483.0] [1541801680.8855941]
[1541801700613.0] [1541801680.953341]
[1541801700614.0] [1541801680.983377]
[1541801700614.0] [1541801681.020771]
[1541801700704.0] [1541801681.051858]
[1541801700705.0] [1541801681.1219919]
[1541801700771.0] [1541801681.151074]
[1541801700828.0] [1541801681.1790879]
[1541801700828.0] [1541801681.21587]
[1541801700901.0] [1541801681.28598]
[1541801700902.0] [1541801681.317869]
[1541801701023.0] [1541801681.344071]
[1541801701024.0] [1541801681.383262]
[1541801701187.0] [1541801681.4429069]
[1541801701187.0] [1541801681.481268]
[1541801701188.0] [1541801681.512629]
[1541801701274.0] [1541801681.553926]
[1541801701369.0] [1541801681.612791]
[1541801701370.0] [1541801681.63961]
[1541801701478.0] [1541801681.6807811]
[1541801701478.0] [1541801681.713713]
[154180170

[] [1541801732.696678]
[] [1541801732.723692]
[] [1541801732.794037]
[] [1541801732.8221362]
[] [1541801732.863418]
[] [1541801732.893196]
[] [1541801732.9636018]
[] [1541801732.992765]
[] [1541801733.0209029]
[] [1541801733.093049]
[] [1541801733.1231549]
[] [1541801733.163371]
[] [1541801733.1892512]
[] [1541801733.246866]
[] [1541801733.287607]
[] [1541801733.328853]
[] [1541801733.3550122]
[] [1541801733.425246]
[] [1541801733.455404]
[] [1541801733.482226]
[] [1541801733.520637]
[] [1541801733.577208]
[] [1541801733.620932]
[] [1541801733.645717]
[] [1541801733.689733]
[] [1541801733.745358]
[] [1541801733.785287]
[] [1541801733.816273]
[] [1541801733.856092]
[] [1541801733.9138281]
[] [1541801733.9409559]
[] [1541801733.9811819]
[] [1541801734.0509949]
[] [1541801734.077328]
[] [1541801734.10561]
[] [1541801734.1472619]
[] [1541801734.206152]
[] [1541801734.2483609]
[] [1541801734.27506]
[] [1541801734.316826]
[] [1541801734.373641]
[] [1541801734.413889]
[] [1541801734.4470122]


...

[] [1541800998.926083]
[1541800998952.0, 1541800998959.0, 1541800998963.0] [1541800998.983583]
[1541800999062.0] [1541800999.021951]
[1541800999069.0] [1541800999.060122]
[1541800999205.0] [1541800999.087289]
[1541800999209.0] [1541800999.155655]
[1541800999216.0] [1541800999.184371]
[1541800999359.0] [1541800999.226623]
[1541800999361.0] [1541800999.254199]
[1541800999438.0] [1541800999.324918]
[1541800999441.0] [1541800999.354738]
[1541800999487.0] [1541800999.392114]
[1541800999612.0] [1541800999.4224699]
[1541800999614.0] [1541800999.488173]
[1541800999706.0] [1541800999.516621]
[1541800999708.0] [1541800999.557487]
[1541800999826.0] [1541800999.5874429]
[1541800999830.0] [1541800999.645834]
[1541800999970.0] [1541800999.682855]
[1541800999971.0] [1541800999.711318]
[1541800999972.0] [1541800999.783573]
[1541801000099.0] [1541800999.810605]
[1541801000103.0] [1541800999.848968]
[1541801000105.0] [1541800999.876724]
[1541801000189.0] [1541800999.947176]
[1541801000192.0] [1541800999

[1541801075386.0] [1541801057.032778]
[1541801075454.0] [1541801057.072761]
[1541801075454.0] [1541801057.102444]
[1541801075455.0] [1541801057.129177]
[1541801075517.0] [1541801057.197521]
[1541801075518.0] [1541801057.237823]
[1541801075518.0] [1541801057.262125]
[1541801075603.0] [1541801057.29967]
[1541801075604.0] [1541801057.358519]
[1541801075605.0] [1541801057.398881]
[1541801075724.0] [1541801057.4261708]
[1541801075725.0] [1541801057.4665291]
[1541801075809.0] [1541801057.524907]
[1541801075810.0] [1541801057.564057]
[1541801075811.0] [1541801057.592833]
[1541801075927.0] [1541801057.662706]
[1541801075927.0] [1541801057.693272]
[1541801076070.0] [1541801057.735146]
[1541801076071.0] [1541801057.76053]
[1541801076185.0] [1541801057.83268]
[1541801076186.0] [1541801057.859193]
[1541801076316.0] [1541801057.898057]
[1541801076317.0] [1541801057.925461]
[1541801076317.0] [1541801057.9950058]
[1541801076443.0] [1541801058.025325]
[1541801076444.0] [1541801058.053239]
[15418010765

[] [1541801109.045558]
[] [1541801109.0737972]
[] [1541801109.112454]
[] [1541801109.1718311]
[] [1541801109.211958]
[] [1541801109.239276]
[] [1541801109.2720268]
[] [1541801109.339095]
[] [1541801109.368931]
[] [1541801109.4049711]
[] [1541801109.435204]
[] [1541801109.5070689]
[] [1541801109.533498]
[] [1541801109.574968]
[] [1541801109.603452]
[] [1541801109.662085]
[] [1541801109.702364]
[] [1541801109.7317498]
[] [1541801109.806293]
[] [1541801109.834391]
[] [1541801109.865574]
[] [1541801109.905062]
[] [1541801109.965549]
[] [1541801109.9953852]
[] [1541801110.035831]
[] [1541801110.065066]
[] [1541801110.125268]
[] [1541801110.160444]
[] [1541801110.200743]
[] [1541801110.228703]
[] [1541801110.300761]
[] [1541801110.329375]
[] [1541801110.369586]
[] [1541801110.399429]
[] [1541801110.458292]
[] [1541801110.4966989]
[] [1541801110.529706]
[] [1541801110.5543702]
[] [1541801110.6238148]
[] [1541801110.662366]
[] [1541801110.69073]
[] [1541801110.7616441]
[] [1541801110.789651]
[

[1541800593447.0] [1541800586.440126]
[1541800593561.0] [1541800586.508276]
[1541800593562.0] [1541800586.548427]
[1541800593563.0] [1541800586.573994]
[1541800593680.0] [1541800586.612436]
[1541800593680.0] [1541800586.681439]
[1541800593803.0] [1541800586.709582]
[1541800593803.0] [1541800586.735856]
[1541800593933.0] [1541800586.8060899]
[1541800593934.0] [1541800586.844609]
[1541800593934.0] [1541800586.8704169]
[1541800594012.0] [1541800586.908787]
[1541800594012.0] [1541800586.9793391]
[1541800594107.0] [1541800587.0068638]
[1541800594108.0] [1541800587.0360901]
[1541800594220.0] [1541800587.074079]
[1541800594221.0] [1541800587.1423712]
[1541800594221.0] [1541800587.169118]
[1541800594361.0] [1541800587.209464]
[1541800594362.0] [1541800587.234674]
[1541800594421.0] [1541800587.303466]
[1541800594421.0] [1541800587.332777]
[1541800594535.0] [1541800587.3692129]
[1541800594536.0] [1541800587.406115]
[1541800594611.0] [1541800587.4662201]
[1541800594612.0] [1541800587.503659]
[154

[1541800657033.0] [1541800638.4147022]
[1541800657100.0] [1541800638.485663]
[1541800657174.0] [1541800638.514355]
[1541800657174.0] [1541800638.5534961]
[1541800657348.0] [1541800638.582062]
[1541800657349.0] [1541800638.650059]
[1541800657349.0] [1541800638.677931]
[1541800657530.0] [1541800638.7155142]
[1541800657530.0] [1541800638.7437592]
[1541800657531.0] [1541800638.813986]
[1541800657601.0] [1541800638.8421981]
[1541800657601.0] [1541800638.880836]
[1541800657686.0] [1541800638.9099278]
[1541800657687.0] [1541800638.976937]
[1541800657801.0] [1541800639.016605]
[1541800657802.0] [1541800639.045019]
[1541800657850.0] [1541800639.115885]
[1541800657924.0] [1541800639.143669]
[1541800657925.0] [1541800639.183541]
[1541800658009.0] [1541800639.21055]
[1541800658010.0] [1541800639.273424]
[1541800658100.0] [1541800639.3101811]
[1541800658258.0] [1541800639.339602]
[1541800658260.0] [1541800639.379155]
[1541800658260.0] [1541800639.4398541]
[1541800658345.0] [1541800639.479288]
[1541

E20038EBB5953DC9B7693E9C
E20038EBB5953F49B7693EA2